In [ ]:
!pip install pyspark==3.5.3

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #3")
conf.set("spark.master", "local[*]")

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

In [ ]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/transfer_cost.txt

--2025-02-07 06:29:05--  https://s3-geospatial.s3.us-west-2.amazonaws.com/transfer_cost.txt
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.153.82, 3.5.80.194, 3.5.79.176, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.153.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286779 (280K) [text/plain]
Saving to: ‘transfer_cost.txt’

transfer_cost.txt   100%[===================>] 280.06K  1.60MB/s    in 0.2s    

2025-02-07 06:29:06 (1.60 MB/s) - ‘transfer_cost.txt’ saved [286779/286779]



In [ ]:
!ls -tl

total 288
drwxr-xr-x 1 root root   4096 Jan  6 14:19 sample_data
-rw-r--r-- 1 root root 286779 Apr 24  2022 transfer_cost.txt


In [ ]:
!head -5 transfer_cost.txt

On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today
On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today


In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

schema = StructType([ StructField("text", StringType(), True)])
transfer_cost_df = spark.read.schema(schema).text("transfer_cost.txt")

In [ ]:
transfer_cost_df.show(truncate=False)

+---------------------------------------------------------------------------+
|text                                                                       |
+---------------------------------------------------------------------------+
|On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85012 is $18.98 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85013 is 26.64 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85020 is 26.34 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85021 is $20.15 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85002 to 85001 is 21.57 at 

In [ ]:
from pyspark.sql.functions import *
regex_str = r'On (\S+) the cost per ton from (\d+) to (\d+) is (\S+) at (.*)'

df_with_new_columns = transfer_cost_df\
    .withColumn('week', regexp_extract('text', regex_str, 1))\
    .withColumn('departure_zipcode', regexp_extract(column('text'), regex_str, 2))\
    .withColumn('arrival_zipcode', regexp_extract(transfer_cost_df.text, regex_str, 3))\
    .withColumn('cost', regexp_extract(col('text'), regex_str, 4))\
    .withColumn('vendor', regexp_extract(col('text'), regex_str, 5))

In [ ]:
df_with_new_columns.printSchema()

root
 |-- text: string (nullable = true)
 |-- week: string (nullable = true)
 |-- departure_zipcode: string (nullable = true)
 |-- arrival_zipcode: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- vendor: string (nullable = true)



In [ ]:
final_df = df_with_new_columns.drop("text")

In [ ]:
final_df.write.csv("extracted.csv")

In [ ]:
!ls -tl

total 292
drwxr-xr-x 2 root root   4096 Feb  7 06:33 extracted.csv
drwxr-xr-x 1 root root   4096 Jan  6 14:19 sample_data
-rw-r--r-- 1 root root 286779 Apr 24  2022 transfer_cost.txt


In [ ]:
!ls -tl extracted.csv/

total 156
-rw-r--r-- 1 root root      0 Feb  7 06:33 _SUCCESS
-rw-r--r-- 1 root root 156423 Feb  7 06:33 part-00000-46eb7828-e77f-4607-88d8-740d7c50069b-c000.csv


In [ ]:
!head -5 extracted.csv/part-00000-a909db0a-d743-4a0c-96fc-60c1a8eef076-c000.csv

head: cannot open 'extracted.csv/part-00000-a909db0a-d743-4a0c-96fc-60c1a8eef076-c000.csv' for reading: No such file or directory


In [ ]:
final_df.write.format("json").save("extracted.json")

In [ ]:
!ls -tl extracted.json/

total 428
-rw-r--r-- 1 root root      0 Feb  7 06:36 _SUCCESS
-rw-r--r-- 1 root root 436305 Feb  7 06:36 part-00000-f594a95a-68fe-4e16-b7ce-db35467f8b36-c000.json


In [ ]:
!head -1 extracted.json/part-00000-104f95b9-f2c6-4f77-a170-583c78106e11-c000.json

head: cannot open 'extracted.json/part-00000-104f95b9-f2c6-4f77-a170-583c78106e11-c000.json' for reading: No such file or directory
